In [1]:
import numpy as np
import pandas as pd
from preprocess_ztf import ZTFLightcurvePreprocessor
from ztf_feature_extractor import ZTFFeatureExtractor

In [2]:
is_training = False
if is_training:
    lc_filename = 'training_lightcurves.parquet'
    features_filename = 'training_features.parquet'
else:
    lc_filename = 'test_lightcurves.parquet'
    features_filename = 'test_features.parquet'

In [3]:
lightcurves = pd.read_parquet(lc_filename)
lightcurves.set_index('oid', inplace=True)
lightcurves

,tid,mjd,candid,fid,pid,diffmaglim,isdiffpos,nid,distnr,magpsf,...,rfid,has_stamp,corrected,dubious,step_id_corr,phase,parent_candid,magpsf_corr,sigmapsf_corr,sigmapsf_corr_ext
oid,,,,,,,,,,,,,,,,,,,,,
ZTF22abtvgfs,ztf,59898.206632,2144206631215015006,1,2144206631215,20.638058,1,2144,3.059537,20.510626,...,649120112.0,True,False,False,1.1.6,0.0,NaN,NaN,NaN,NaN
ZTF22abtvgfs,ztf,59900.201505,2146201501215015005,1,2146201501215,20.234230,1,2146,2.772549,19.762703,...,649120112.0,True,False,False,1.1.6,0.0,NaN,NaN,NaN,NaN
ZTF22abtvgfs,ztf,59902.164630,2148164621215015003,2,2148164621215,20.181196,1,2148,3.044964,19.821419,...,649120212.0,True,False,False,1.1.6,0.0,NaN,NaN,NaN,NaN
ZTF22abtvgfs,ztf,59902.309224,2148309221215015003,1,2148309221215,20.046690,1,2148,2.968886,19.546738,...,649120112.0,True,False,False,1.1.6,0.0,NaN,NaN,NaN,NaN
ZTF22abtvgfs,ztf,59904.179259,2150179251215015001,1,2150179251215,20.677929,1,2150,2.896441,19.297873,...,649120112.0,True,False,False,1.1.6,0.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF18aaxmixe,ztf,59290.421991,1536421996115010008,1,1536421996115,20.565900,-1,1536,1.108910,20.045800,...,NaN,False,True,False,correction_0.0.1,0.0,NaN,18.735037,0.069488,0.086767
ZTF18aaxmixe,ztf,59292.434444,1538434446115010011,1,1538434446115,20.469616,-1,1538,0.392978,20.121468,...,478120161.0,True,True,False,correction_0.0.1,0.0,NaN,18.713398,0.020706,0.054983
ZTF18aaxmixe,ztf,59309.337917,1555337916115010007,1,1555337916115,20.539300,-1,1555,0.627728,20.265300,...,NaN,False,True,False,correction_0.0.1,0.0,NaN,18.677181,0.017573,0.052305


In [4]:
preprocessor = ZTFLightcurvePreprocessor()
detections = preprocessor.preprocess(lightcurves)

In [5]:
oids = detections.index.unique()
oids

Index(['ZTF18aaaqexr', 'ZTF18aaceepu', 'ZTF18aafpfwr', 'ZTF18aahshhp',
       'ZTF18aahtjsc', 'ZTF18aaisqmw', 'ZTF18aaizerg', 'ZTF18aakfabl',
       'ZTF18aakkrjm', 'ZTF18aaqvuhl',
       ...
       'ZTF22abquwiv', 'ZTF22abqxkhd', 'ZTF22abrredj', 'ZTF22abseymh',
       'ZTF22abswzyp', 'ZTF22absxxub', 'ZTF22abtciik', 'ZTF22abtctvo',
       'ZTF22abtootl', 'ZTF22abtvgfs'],
      dtype='object', name='oid', length=931)

In [6]:
feature_extractor = ZTFFeatureExtractor(bands=(1, 2))

In [7]:
# We use the compute_features method
non_detections = pd.DataFrame()
features = feature_extractor.compute_features(
    detections = detections, #.loc[oids[:2]],
    non_detections = non_detections)

 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda

 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda

 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda

 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda3/envs/alerce/lib/python3.8/site-packages/scipy/optimize/minpack.py:828: OptimizeWarning:Covariance of the parameters could not be estimated
 /home/ireyes/anaconda

In [8]:
# The result is a dataframe with the features
features

,Amplitude_1,Amplitude_2,AndersonDarling_1,AndersonDarling_2,Autocor_length_1,Autocor_length_2,Beyond1Std_1,Beyond1Std_2,Con_1,Con_2,...,n_neg_1,n_neg_2,n_non_det_after_fid_1,n_non_det_after_fid_2,n_non_det_before_fid_1,n_non_det_before_fid_2,n_pos_1,n_pos_2,positive_fraction_1,positive_fraction_2
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18aaaqexr,0.448069,NaN,0.359010,NaN,2.0,NaN,0.444444,NaN,0.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,9.0,2.0,1.0,1.000
ZTF18aaceepu,1.281059,1.069200,1.000000,1.000000,5.0,7.0,0.307692,0.185185,0.0,0.096154,...,0.0,0.0,0.0,0.0,0.0,0.0,26.0,54.0,1.0,1.000
ZTF18aafpfwr,0.597358,0.426465,0.908926,0.446642,1.0,2.0,0.166667,0.285714,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,7.0,1.0,1.000
ZTF18aahshhp,0.619962,0.419447,0.999998,0.999654,1.0,1.0,0.111111,0.125000,0.0,0.000000,...,0.0,1.0,0.0,0.0,0.0,0.0,9.0,7.0,1.0,0.875
ZTF18aahtjsc,0.319470,0.481995,0.796648,0.995791,2.0,1.0,0.250000,0.125000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,8.0,1.0,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZTF22absxxub,0.529666,0.510866,0.999132,0.989501,1.0,1.0,0.166667,0.125000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,6.0,8.0,1.0,1.000
ZTF22abtciik,0.702260,0.348016,0.977328,0.944155,2.0,3.0,0.166667,0.187500,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,12.0,16.0,1.0,1.000
ZTF22abtctvo,0.370568,0.492037,0.688741,0.676569,2.0,2.0,0.142857,0.222222,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,7.0,9.0,1.0,1.000


In [9]:
features.to_parquet(features_filename)